<a href="https://colab.research.google.com/github/thxsxth/RLMimic/blob/master/Vaso_Fluid_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd 'drive/My Drive/sepsis3-cohort'

/content/drive/My Drive/sepsis3-cohort


# This NoteBook will Describe Cohort selection, and final data preperation. 

Whilst it is rather long you can open it in Colab and choose the respective section from the table to conetents to navigate

In [ ]:
import torch
import numpy as np
import pandas as pd
import datetime as dt
import random
import time
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import os
import glob
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence,pad_packed_sequence

## Making the necessary Imports

### Vitals, Sofa, Labs are directly imported from Big Query
vaso_df is derived so that we have starttime and end time of Vasopressor Durations broken into hourly intervals


In [ ]:
vitals=pd.read_csv('./Vitals/Vitals.csv',parse_dates=['charttime']) #pivoted vitals
sofa=pd.read_csv('./pivoted_sofa/pivoted_sofa.csv',parse_dates=['endtime','starttime']) #pivoted sofa
# sofa_vaso=pd.read_csv('./pivoted_sofa/sofa_vaso.csv',parse_dates=['endtime','starttime']) 
labs=pd.read_csv('./pivoted_labs/Pivoted_labs.csv',parse_dates=['charttime'])
vaso_df=pd.read_csv('Vaso_full.csv',parse_dates=['endtime','starttime'])

print('DONE')

DONE


In [ ]:
# vaso=pd.read_csv('./pivoted_sofa/vasopressin.csv',parse_dates=['endtime','starttime']) 
# print("DONE")

DONE


### Now we will merge using pd.merge so that sofa will contain the rates rate_vasopressin','rate_phenylephrine in addition to what it already had

In [ ]:
sofa_merged=sofa.merge(vaso_df[['icustay_id','starttime','rate_vasopressin','rate_phenylephrine']],on=['icustay_id','starttime'],how='outer')
sofa_merged.to_csv('sofa_with_vaso3.csv',index=False)
print('Done')

#### Now let's check if this had worked as expected

In [ ]:
sofa_merged[sofa_merged.icustay_id==200024]

In [ ]:
pat=np.random.choice(list(sofa.icustay_id))
# pat=200030
sofa[sofa.icustay_id==pat].head()

,icustay_id,hr,starttime,endtime,PaO2FiO2Ratio_novent,PaO2FiO2Ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,MeanBP_min,GCS_min,urineoutput,bilirubin_max,creatinine_max,platelet_min,respiration,coagulation,liver,cardiovascular,cns,renal,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,SOFA_24hours
2813598,252135,0,2166-04-09 09:00:00,2166-04-09 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,2,2
2813599,252135,1,2166-04-09 10:00:00,2166-04-09 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,90.666702,15.0,600.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,0,0,0,0,2,2
2813600,252135,2,2166-04-09 11:00:00,2166-04-09 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,74.333298,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,0,0,0,0,2,2
2813601,252135,3,2166-04-09 12:00:00,2166-04-09 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,79.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0,0,0,0,0,2,2
2813602,252135,4,2166-04-09 13:00:00,2166-04-09 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,74.666702,15.0,120.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0,0,0,0,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2813840,252135,242,2166-04-19 11:00:00,2166-04-19 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,0,1,0,2,3
2813841,252135,243,2166-04-19 12:00:00,2166-04-19 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,65.333298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0.0,0,0,0,1,0,2,3
2813842,252135,244,2166-04-19 13:00:00,2166-04-19 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,0,1,0,2,3
2813843,252135,245,2166-04-19 14:00:00,2166-04-19 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,0,1,0,2,3


In [ ]:
# pat=200030
sofa_merged[sofa_merged.icustay_id==pat].head(25)

,icustay_id,hr,starttime,endtime,PaO2FiO2Ratio_novent,PaO2FiO2Ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,MeanBP_min,GCS_min,urineoutput,bilirubin_max,creatinine_max,platelet_min,respiration,coagulation,liver,cardiovascular,cns,renal,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,SOFA_24hours,rate_vasopressin,rate_phenylephrine,Vaso
742,200030,0.0,2150-11-13 13:00:00,2150-11-13 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,76.000000,NaN,NaN,NaN,1.4,139.0,NaN,1.0,NaN,0.0,NaN,1.0,0.0,1.0,2.0,0.0,0.0,1.0,4.0,NaN,NaN,0.00
743,200030,1.0,2150-11-13 14:00:00,2150-11-13 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,63.666698,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,0.0,1.0,2.0,1.0,0.0,1.0,5.0,NaN,NaN,0.00
744,200030,2.0,2150-11-13 15:00:00,2150-11-13 16:00:00,NaN,NaN,NaN,0.08,NaN,NaN,72.666702,NaN,720.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,0.0,0.0,1.0,2.0,3.0,0.0,1.0,7.0,NaN,NaN,0.08
745,200030,3.0,2150-11-13 16:00:00,2150-11-13 17:00:00,NaN,NaN,NaN,0.08,NaN,NaN,69.333298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,0.0,0.0,1.0,2.0,3.0,0.0,1.0,7.0,NaN,NaN,0.08
746,200030,4.0,2150-11-13 17:00:00,2150-11-13 18:00:00,NaN,NaN,NaN,0.08,NaN,NaN,64.000000,NaN,360.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,0.0,0.0,1.0,2.0,3.0,0.0,1.0,7.0,NaN,NaN,0.08
747,200030,5.0,2150-11-13 18:00:00,2150-11-13 19:00:00,NaN,NaN,NaN,0.08,NaN,NaN,77.000000,15.0,360.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,1.0,2.0,3.0,0.0,1.0,7.0,NaN,NaN,0.08
748,200030,6.0,2150-11-13 19:00:00,2150-11-13 20:00:00,NaN,NaN,NaN,0.08,NaN,8.0,75.000000,NaN,240.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,0.0,0.0,1.0,2.0,3.0,0.0,1.0,7.0,NaN,NaN,0.88
749,200030,7.0,2150-11-13 20:00:00,2150-11-13 21:00:00,NaN,NaN,NaN,0.08,NaN,5.0,82.000000,15.0,760.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.0,0.0,0.0,1.0,2.0,3.0,0.0,1.0,7.0,NaN,NaN,0.58
750,200030,8.0,2150-11-13 21:00:00,2150-11-13 22:00:00,NaN,NaN,NaN,0.05,NaN,5.0,82.000000,NaN,180.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,0.0,0.0,1.0,2.0,3.0,0.0,1.0,7.0,NaN,NaN,0.55
751,200030,9.0,2150-11-13 22:00:00,2150-11-13 23:00:00,NaN,NaN,NaN,0.08,NaN,5.0,86.666702,15.0,260.0,NaN,NaN,171.0,NaN,0.0,NaN,3.0,0.0,0.0,0.0,1.0,2.0,3.0,0.0,1.0,7.0,NaN,NaN,0.58


In [ ]:
sofa_=sofa_merged
sofa_.head()

,icustay_id,hr,starttime,endtime,PaO2FiO2Ratio_novent,PaO2FiO2Ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,MeanBP_min,GCS_min,urineoutput,bilirubin_max,creatinine_max,platelet_min,respiration,coagulation,liver,cardiovascular,cns,renal,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,SOFA_24hours,rate_vasopressin,rate_phenylephrine
0,200001,0.0,2181-11-25 18:00:00,2181-11-25 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,2.0,3.0,NaN,NaN
1,200001,1.0,2181-11-25 19:00:00,2181-11-25 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,76.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,2.0,3.0,NaN,NaN
2,200001,2.0,2181-11-25 20:00:00,2181-11-25 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,2.0,3.0,NaN,NaN
3,200001,3.0,2181-11-25 21:00:00,2181-11-25 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,2.0,3.0,NaN,NaN
4,200001,4.0,2181-11-25 22:00:00,2181-11-25 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,71.0,NaN,250.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,3.0,0.0,1.0,0.0,0.0,0.0,3.0,4.0,NaN,NaN


In [ ]:
sofa_['Vaso']=sofa_['rate_norepinephrine'].add(0.1*sofa_['rate_dopamine'],fill_value=0).add(sofa_['rate_epinephrine'],fill_value=0)
sofa_['Vaso']=sofa_['Vaso'].add(0.1*sofa_['rate_dobutamine'],fill_value=0).add(10*sofa_['rate_vasopressin'],fill_value=0).fillna(0).add(
    0.09*sofa_['rate_phenylephrine'],fill_value=0)

sofa_.head()


,icustay_id,hr,starttime,endtime,PaO2FiO2Ratio_novent,PaO2FiO2Ratio_vent,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,MeanBP_min,GCS_min,urineoutput,bilirubin_max,creatinine_max,platelet_min,respiration,coagulation,liver,cardiovascular,cns,renal,respiration_24hours,coagulation_24hours,liver_24hours,cardiovascular_24hours,cns_24hours,renal_24hours,SOFA_24hours,rate_vasopressin,rate_phenylephrine,Vaso
0,200001,0.0,2181-11-25 18:00:00,2181-11-25 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,2.0,3.0,NaN,NaN,0.0
1,200001,1.0,2181-11-25 19:00:00,2181-11-25 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,76.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,2.0,3.0,NaN,NaN,0.0
2,200001,2.0,2181-11-25 20:00:00,2181-11-25 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,2.0,3.0,NaN,NaN,0.0
3,200001,3.0,2181-11-25 21:00:00,2181-11-25 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,1.0,0.0,0.0,0.0,2.0,3.0,NaN,NaN,0.0
4,200001,4.0,2181-11-25 22:00:00,2181-11-25 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,71.0,NaN,250.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,3.0,0.0,1.0,0.0,0.0,0.0,3.0,4.0,NaN,NaN,0.0


In [9]:
# sofa_.to_csv('sofa_with_vaso3.csv',index=False)
print('DONE')

DONE


In [16]:
sofa_[sofa_.Vaso!=0][['rate_vasopressin','rate_phenylephrine','rate_epinephrine',
                     'rate_norepinephrine','rate_dopamine','rate_dobutamine','Vaso']]

,rate_vasopressin,rate_phenylephrine,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,Vaso
98,NaN,0.5,NaN,NaN,NaN,NaN,0.045
99,NaN,1.0,NaN,NaN,NaN,NaN,0.090
100,NaN,0.5,NaN,NaN,NaN,NaN,0.045
101,NaN,0.5,NaN,NaN,NaN,NaN,0.045
102,NaN,0.5,NaN,NaN,NaN,NaN,0.045
...,...,...,...,...,...,...,...
6885728,0.02,NaN,NaN,NaN,NaN,NaN,0.200
6885729,0.02,NaN,NaN,NaN,NaN,NaN,0.200
6933421,NaN,36.0,NaN,NaN,NaN,NaN,3.240
6933422,NaN,36.0,NaN,NaN,NaN,NaN,3.240


In [ ]:
co=pd.read_csv('sepsis3_adults.csv',
               parse_dates=['intime','outtime','suspected_infection_time_poe']) #cohort + demographics
co=co.set_index('icustay_id')

admissions=pd.read_csv('admissions.csv',parse_dates=['ADMITTIME','DISCHTIME','DEATHTIME'])
admissions=admissions.set_index('icustay_id').sort_index()
co['death_time']=admissions['DEATHTIME']
print('Done')

Done


### Fluids from CV and MetaVision Csv files, (Note that these are already cleaned so the 'tev' gives total equivalent Volume and the SQL code is available under SQL folder)



In [ ]:
input_cv=pd.read_csv('./Fluids/cleaned_input_cv.csv',parse_dates=['charttime']) 
input_mv=pd.read_csv('./Fluids/input_eventsMV.csv',parse_dates=['starttime','endtime'])
input_cv=input_cv[['icustay_id','charttime','tev']]
input_mv=input_mv[['icustay_id','endtime','tev']]
input_mv['tev_mv']=input_mv['tev']
input_mv['charttime']=input_mv['endtime']
input_mv=input_mv.drop('tev',axis=1)
input_fluids=input_mv.merge(input_cv,on=['icustay_id','charttime'],how='outer')[['icustay_id','charttime','tev','tev_mv']]
input_fluids['tev'],input_fluids['tev_mv']=input_fluids['tev'].fillna(0),input_fluids['tev_mv'].fillna(0)
input_fluids['volume']=input_fluids['tev']+input_fluids['tev_mv']
input_fluids=input_fluids.drop(['tev','tev_mv'],axis=1)
input_fluids.head()

,icustay_id,charttime,volume
0,200001,2181-11-25 22:55:00,50.000
1,200001,2181-11-27 15:47:00,200.000
2,200001,2181-11-27 23:26:00,250.000
3,200010,2132-08-05 02:10:00,17.708
4,200010,2132-08-05 01:36:00,1000.000


## Helper Functions

In [ ]:
def get_pat_dfs(pat):

  pat_sofa=sofa[sofa.icustay_id==pat]
  pat_vitals=vitals[vitals.icustay_id==pat]
  # pat_labs=labs[labs.icustay_id==pat]
  pat_df=pd.concat([pat_vitals.set_index('charttime')
                              ,
                              pat_sofa.set_index('endtime')]).resample('H').last()[['HeartRate','SysBP','DiasBP',	'MeanBP','RespRate','SpO2',
                                      'liver_24hours','cardiovascular_24hours',
                                      'cns_24hours','renal_24hours','SOFA_24hours']].resample('H').last()
  
  if co.loc[pat].HOSPITAL_EXPIRE_FLAG==1:
    pat_df=pat_df.truncate(after=co.loc[pat].death_time)

  return pat_sofa,pat_vitals,pat_df #pat_labs

print('Done')


Done


### Checking if Death Time<ICU off time

In [ ]:
non_icu_deaths=0
non_icu_pats=[]
for pat in co[co.death_time.notna()].index:
  pat_sofa,pat_vitals,pat_labs,pat_df=get_pat_dfs(pat)
  if pat_df.shape[0]<1:
    continue
  if pat_df.index[-1]+pd.Timedelta('1 Hour')<co.loc[pat].death_time:
    non_icu_deaths+=1
    print('Last Vital time :',pat_df.index[-1],'Death :',co.loc[pat].death_time,'cohort out time ',co.loc[pat].outtime)
    non_icu_pats.append(pat)





In [ ]:
# lens=[]
# maxna=[]
# for (pat,_) in sofa_len_list[500:]:
#   pat_sofa=sofa[sofa.icustay_id==pat]
#   pat_vitals=vitals[vitals.icustay_id==pat]
#   pat_labs=labs[labs.icustay_id==pat]
#   temp=pd.concat([pat_vitals.set_index('charttime')
#                               ,
#                               pat_sofa.set_index('endtime')]).resample('H').last()[['HeartRate','SysBP','DiasBP',	'MeanBP','RespRate','SpO2',
#                                       'liver_24hours','cardiovascular_24hours',
#                                       'cns_24hours','renal_24hours','SOFA_24hours']]
#   na_=temp.isna().sum(axis=0).max()
#   length=temp.dropna().shape[0]
  
#   print('patient number : ',pat, 'length of the trajectory is :  ',temp.shape[0],' number of NA hours are : ', na_,' After dropping NA :',length)
#   lens.append(length)
#   maxna.append(na_)


In [ ]:
len(lens)

In [ ]:
pats=(sofa_len_list[500:])

In [ ]:
full_lens=[x[1] for x in pats]
patient_ids=[x[0] for x in pats]

In [ ]:
NaDf=pd.DataFrame([full_lens,maxna,lens,patient_ids]).T
NaDf.columns=['lengths','NA','Reduced_lens','Pat_ID']

In [ ]:
NaDf.head()

In [ ]:
NaDf['na_per']=NaDf['NA']/NaDf['lengths']

In [ ]:
NaDf.describe()

In [ ]:
NaDf[NaDf.na_per<0.2].shape

In [ ]:
# NaDf.to_csv('NA_info.csv')

NaDf=pd.read_csv('NA_info.csv')
NaDf.head()

## Settling on a Cohort
Let's  stick to NA%<0.2 and #NA<15

In [ ]:
len(set(co[co.death_time.notna()].index).intersection(NaDf[NaDf.na_per<0.2][NaDf.NA<15].Pat_ID))

### Let's do the same as above for deaths

In [ ]:
lens=[]
maxna=[]
for pat in co[co.death_time.notna()].index:
  pat_sofa=sofa[sofa.icustay_id==pat]
  pat_vitals=vitals[vitals.icustay_id==pat]
  pat_labs=labs[labs.icustay_id==pat]
  temp=pd.concat([pat_vitals.set_index('charttime')
                              ,
                              pat_sofa.set_index('endtime')]).resample('H').last()[['HeartRate','SysBP','DiasBP',	'MeanBP','RespRate','SpO2',
                                      'liver_24hours','cardiovascular_24hours',
                                      'cns_24hours','renal_24hours','SOFA_24hours']]
  temp=temp.truncate(after=co.loc[pat].death_time)
  na_=temp.isna().sum(axis=0).max()
  length=temp.dropna().shape[0]
  
  print('patient number : ',pat, 'length of the trajectory is :  ',temp.shape[0],' number of NA hours are : ', na_,' After dropping NA :',length)
  lens.append(length)
  maxna.append(na_)

In [ ]:
NaDf_deaths=pd.DataFrame([maxna,lens,list(co[co.death_time.notna()].index)]).T

In [ ]:
NaDf_deaths.columns=['NA','Reduced_lens','Pat_ID']
NaDf_deaths['lengths']=NaDf_deaths.NA+NaDf_deaths.Reduced_lens
NaDf_deaths['na_per']=NaDf_deaths.NA/NaDf_deaths.lengths

In [ ]:
NaDf_deaths[NaDf_deaths.na_per<0.2]

### Here let's consider the following patients for our analysis death

In [ ]:
len(set(NaDf_deaths[NaDf_deaths['na_per']<0.2][NaDf_deaths.NA<20][NaDf_deaths.lengths>3].Pat_ID)-set(non_icu_pats))

In [ ]:
dead_pats=list(set(NaDf_deaths[NaDf_deaths['na_per']<0.2][NaDf_deaths.NA<20][NaDf_deaths.lengths>3].Pat_ID)-set(non_icu_pats))

In [ ]:
NaDf_deaths.to_csv('Na_df_deaths.csv')

In [ ]:
pat_sofa,pat_vitals,pat_labs,pat_df=get_pat_dfs(200035)

In [ ]:
pat=200035
age,weight=co.loc[pat].age,co.loc[pat].Weight
# pat_vitals['gender']=co.loc[pat].is_male
pat_vitals.['Weight']=weight
pat_vitals['age']=age


The following csv files give information on Missing values %s and deaths

In [ ]:
NaDf_deaths=pd.read_csv('Na_df_deaths.csv')
NaDf=pd.read_csv('NA_info.csv')

In [ ]:
surv_pats=NaDf[NaDf.Reduced_lens>23][NaDf.na_per<0.2].Pat_ID.values
print(len(surv_pats))

pats=NaDf_deaths[NaDf_deaths.Reduced_lens>23][NaDf_deaths.na_per<0.2].Pat_ID.values
print(len(pats))

We also want to consider the weight so figuring out the patients who have a weight recorded

In [ ]:
dead_pats=co.loc[co.index.isin(pats)][co.Weight.notna()].index.values

In [ ]:
len(co.loc[co.index.isin(surv_pats)][co.Weight.notna()].index.values)

In [ ]:
alive_pats=co.loc[co.index.isin(surv_pats)][co.Weight.notna()].index.values

In [ ]:
NaDf_deaths[NaDf_deaths.Reduced_lens>23].Reduced_lens.describe()

In [ ]:
NaDf.sort_values(by='Reduced_lens')

In [ ]:
NaDf[NaDf.Reduced_lens!=0].sort_values(by='Reduced_lens')

In [ ]:
all_pats=set(NaDf[NaDf.na_per<0.2][NaDf.NA<15].Pat_ID).union(set(dead_pats))

In [ ]:
dead_pats=list(set(co[co.HOSPITAL_EXPIRE_FLAG==1].index).intersection(set(all_pats)))

In [ ]:
i=0
bad_list=[]
for pat in list(all_pats):
  pat_sofa,pat_vitals,pat_labs,pat_df=get_pat_dfs(pat)
  # pat_df=pat_df.truncate(before=co.loc[pat].suspected_infection_time_poe)
  if pat_df.shape[0]<4:
    i+=1
    print(pat, i)
    bad_list.append(pat)



In [ ]:
pat_sofa,pat_vitals,pat_labs,pat_df=get_pat_dfs(262158)

In [ ]:
co.columns

# Dataset class

## Re Checking Data

In [ ]:
patient_list=list(np.load('new_pats.npy'))

In [ ]:
len(set(icustays).intersection(set(patient_list)))

9612

In [ ]:
len(patient_list)

19511

In [ ]:
co.loc[patient_list[8]]

Unnamed: 0                                         38207
hadm_id                                           162599
excluded                                               0
intime                               2180-11-18 23:51:36
outtime                              2180-11-20 18:46:11
dbsource                                      metavision
suspected_infection_time_poe         2180-11-18 20:20:00
suspected_infection_time_poe_days                      0
specimen_poe                               BLOOD CULTURE
positiveculture_poe                                    1
antibiotic_time_poe                  2180-11-19 00:00:00
blood_culture_time                   2180-11-18 20:20:00
blood_culture_positive                                 1
age                                                   53
gender                                                 F
is_male                                                0
ethnicity                                 ASIAN - KOREAN
race_white                     

In [ ]:
non_icu_deaths=0
non_icu_pats=[]
for pat in patient_list:
  pat_sofa,pat_vitals,pat_df=get_pat_dfs(pat)
  if pat_df.shape[0]<1:
    continue
  if pat_df.index[-1]+pd.Timedelta('1 Hour')<co.loc[pat].death_time:
    non_icu_deaths+=1
    print('Last Vital time :',pat_df.index[-1],'Death :',co.loc[pat].death_time,'cohort out time ',co.loc[pat].outtime)
    non_icu_pats.append(pat)



Last Vital time : 2148-12-28 16:00:00 Death : 2148-12-29 00:15:00 cohort out time  2148-12-28 16:16:49
Last Vital time : 2115-10-26 00:00:00 Death : 2115-11-12 18:10:00 cohort out time  2115-10-26 00:25:22
Last Vital time : 2158-08-01 16:00:00 Death : 2158-08-03 19:22:00 cohort out time  2158-08-01 16:58:29
Last Vital time : 2144-09-25 12:00:00 Death : 2144-10-09 06:50:00 cohort out time  2144-09-25 12:46:34
Last Vital time : 2108-04-17 01:00:00 Death : 2108-05-02 05:00:00 cohort out time  2108-04-17 01:51:03
Last Vital time : 2152-12-05 15:00:00 Death : 2153-01-02 18:10:00 cohort out time  2152-12-05 15:15:02
Last Vital time : 2136-02-23 22:00:00 Death : 2136-02-27 10:10:00 cohort out time  2136-02-23 22:07:14
Last Vital time : 2112-02-05 17:00:00 Death : 2112-02-09 14:04:00 cohort out time  2112-02-05 17:25:59
Last Vital time : 2145-09-30 18:00:00 Death : 2145-12-07 10:35:00 cohort out time  2145-09-30 18:18:11
Last Vital time : 2125-04-25 18:00:00 Death : 2125-04-28 08:41:00 cohort 

In [ ]:
RL_Pat_list=np.array(list(set(patient_list)-set(non_icu_pats)))

In [ ]:
RL_Pat_list=list(np.load('RL_Pat_list.npy'))

In [ ]:
non_icu_deaths=0
non_icu_pats=[]
for pat in list(RL_Pat_list):
  pat_sofa,pat_vitals,pat_df=get_pat_dfs(pat)
  if pat_df.shape[0]<1:
    continue
  if pat_df.index[-1]+pd.Timedelta('1 Hour')<co.loc[pat].death_time:
    non_icu_deaths+=1
    print('Last Vital time :',pat_df.index[-1],'Death :',co.loc[pat].death_time,'cohort out time ',co.loc[pat].outtime)
    non_icu_pats.append(pat)


In [ ]:
dead_pats=list(set(co[co.HOSPITAL_EXPIRE_FLAG==1].index).intersection(set(RL_Pat_list)))

In [ ]:
len(dead_pats)

1830

In [ ]:
dead_pats2=list(set(co[co.HOSPITAL_EXPIRE_FLAG==1].index).intersection(set(patient_list)))
len(dead_pats2)

2754

In [ ]:
dead_pats3=list(set(co[co.HOSPITAL_EXPIRE_FLAG==1].index).intersection(set(icustays)))
len(dead_pats3)

1424

In [ ]:
len(icustays)

17083